In [40]:
#Import library
import pandas as pd
import gurobipy as gp 
from gurobipy import GRB
import logging

In [ ]:
# Parameters
years = [1, 2, 3]
skills = ['s1', 's2', 's3']

current_workforce = {'s1': 2000, 's2': 1500, 's3': 1000}
demand = {
    (1, 's1'): 1000,
    (1, 's2'): 1400,
    (1, 's3'): 1000,
    (2, 's1'): 500,
    (2, 's2'): 2000,
    (2, 's3'): 1500,
    (3, 's1'): 0,
    (3, 's2'): 2500,
    (3, 's3'): 2000
}
rookie_attrition = {'s1': 0.25, 's2': 0.20, 's3': 0.10}
veteran_attrition = {'s1': 0.10, 's2': 0.05, 's3': 0.05}
demoted_attrition = 0.50
max_hiring = {
    (1, 's1'): 500,
    (1, 's2'): 800,
    (1, 's3'): 500,
    (2, 's1'): 500,
    (2, 's2'): 800,
    (2, 's3'): 500,
    (3, 's1'): 500,
    (3, 's2'): 800,
    (3, 's3'): 500
}
max_overmanning = 150
max_parttime = 50
parttime_cap = 0.50
max_train_unskilled = 200
max_train_semiskilled = 0.25

training_cost = {'s1': 400, 's2': 500}
layoff_cost = {'s1': 200, 's2': 500, 's3': 500}
parttime_cost = {'s1': 500, 's2': 400, 's3': 400}
overmanning_cost = {'s1': 1500, 's2': 2000, 's3': 3000}

In [43]:
#Import library
import pandas as pd
import numpy as np
import gurobipy as gp 
from gurobipy import GRB
import logging



manpower = gp.Model('Manpower planning')

## --------------------------------------------------- --------------------------------------------------------    ##
#                      Decision variables: Define the decisions variable; all variables are non-negative integers  ##
## --------------------------------------------------- --------------------------------------------------------    ##

hire = manpower.addVars(years, skills, ub=max_hiring,vtype=GRB.INTEGER, name="Hire")
part_time = manpower.addVars(years, skills, ub=max_parttime,name="Part_time")
layoff = manpower.addVars(years, skills, vtype=GRB.INTEGER,name="Layoff")
excess = manpower.addVars(years, skills,vtype=GRB.INTEGER, name="Overmanned")
train = manpower.addVars(years, skills, skills, vtype=GRB.INTEGER, name="Retrain")
workforce = manpower.addVars(years, skills,vtype=GRB.INTEGER, name="Workforce")


## --------------------------------------------------- --------------------------------------------------------    ##
#                      Constraints                                                                                   ##
## --------------------------------------------------- --------------------------------------------------------    ##
#(1)(2)(3). Rescruitment constaints:
Rescruitment = manpower.addConstrs(( hire[year,level] <= max_hiring[year,level] for year in years for level in skills), "Rescruitment")
#(4)(5) Retraining capabilities constrainst:
UnskilledTrain1 = manpower.addConstrs((train[year, 's1', 's2'] <= max_train_unskilled for year in years), "UnskilledTrain1")
UnskilledTrain2 = manpower.addConstrs((train[year, 's1', 's3'] == 0 for year in years), "UnskilledTrain2")
SemiskilledTrain = manpower.addConstrs((train[year,'s2', 's3'] <= max_train_semiskilled * workforce[year,'s3'] for year in years), "Semiskilled_training")
manpower.addConstrs
#(6) Part-time workers constrainst:
PartTime = manpower.addConstrs((part_time[year,level] <= max_overmanning for year in years for level in skills), "PartTime")
#(7)Overmanning:
Overmanning = manpower.addConstrs((excess.sum(year, '*') <= max_overmanning for year in years), "Overmanning")
#(8) Workforce available:
Balance = manpower.addConstrs(
    (workforce[year, level] == (1-veteran_attrition[level])*(current_workforce[level] if year == 1 else workforce[year-1, level])
    + (1-rookie_attrition[level])*hire[year, level] + gp.quicksum((1- veteran_attrition[level])* train[year, level2, level]
                                                        -train[year, level, level2] for level2 in skills if level2 < level)
    + gp.quicksum((1- demoted_attrition)* train[year, level2, level] -train[year, level, level2] for level2 in skills if level2 > level)
    - layoff[year, level] for year in years for level in skills), "Balance")

Demand = manpower.addConstrs((workforce[year, level] ==
     demand[year,level] + excess[year, level] + parttime_cap * part_time[year, level]
                     for year in years for level in skills), "Requirements")


## --------------------------------------------------- --------------------------------------------------------    ##
#                      Objective function Minimize total cost                                                      ##
## --------------------------------------------------- --------------------------------------------------------    ##
obj2 =  gp.quicksum((training_cost[level]*train[year, level, skills[skills.index(level)+1]] if level < 's3' else 0)
                + layoff_cost[level]*layoff[year, level]
                + parttime_cost[level]*part_time[year, level]
                + overmanning_cost[level] * excess[year, level] for year in years for level in skills)
manpower.setObjective(obj2, GRB.MINIMIZE)

manpower.optimize()


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 48 rows, 72 columns and 135 nonzeros
Model fingerprint: 0xca8565d3
Variable types: 9 continuous, 63 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [2e+02, 3e+03]
  Bounds range     [5e+01, 8e+02]
  RHS range        [2e+02, 3e+03]
Presolve removed 34 rows and 25 columns
Presolve time: 0.00s
Presolved: 14 rows, 47 columns, 89 nonzeros
Variable types: 0 continuous, 47 integer (0 binary)

Root relaxation: objective 4.986773e+05, 11 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 498677.285    0    8          - 498677.285  

In [44]:
manpower.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 48 rows, 72 columns and 135 nonzeros
Model fingerprint: 0xca8565d3
Variable types: 9 continuous, 63 integer (0 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+00]
  Objective range  [2e+02, 3e+03]
  Bounds range     [5e+01, 8e+02]
  RHS range        [2e+02, 3e+03]
Presolved: 14 rows, 47 columns, 89 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 2
  MIR: 8
  StrongCG: 3

Explored 1 nodes (62 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 16 (of 16 available processors)

Solution count 10: 508700 509000 509300 ... 532700

Optimal solution found (tolerance 1.00e-04)
Best objective 5.087000000000e+05, best bound 5.087000000000e+05, gap 0.0000%


In [ ]:
## Output ###
rows = years.copy()
columns = skills.copy()
hire_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

layoff_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
parttime_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)
excess_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

def show_solutions(vars_name,output_table, title,arg1,arg2,arg3=None):
    """ 
    This function is to create an output solution results from optimization model
    Args:
    - arg1,args are the parameters (arg1 is the columns of the output data, args is the index of the output data)
    - title is the name of the table for display purpose
    
    """
    import pandas as pd 
    import numpy as np

    for arg1, arg2 in vars_name.keys():
        if (abs(vars_name[arg1, arg2].x) > 1e-6):
            output_table.loc[arg1, arg2] = np.round(vars_name[arg1, arg2].x, 1)
    print("* {} \n ".format(title),output_table)
    print("___________________________________")
    return output_table

hire_plan =show_solutions(hire,hire_plan,"Hiring plan's solution detail","year","level",arg3=None)
layoff_plan =show_solutions(layoff,layoff_plan,"Layoff plan's solution detail","year","level",arg3=None)
parttime_plan =show_solutions(part_time,parttime_plan,"Parttime plan's solution detail","year","level",arg3=None)
excess_plan =show_solutions(excess,excess_plan,"Overmanning plan's solution detail","year","level",arg3=None)



columns_2 = ['{0} to {1}'.format(level1, level2) for level1 in skills for level2 in skills if level1 != level2]
train_plan = pd.DataFrame(columns=columns_2, index=rows, data=0.0)
for year, level1, level2 in train.keys():
    col = '{0} to {1}'.format(level1, level2)
    if (abs(train[year, level1, level2].x) > 1e-6):
        train_plan.loc[year, col] = np.round(train[year, level1, level2].x, 1)
print("* {} \n ".format("* Retraining detail plan:"),train_plan)
print("* Retraining detail plan:")
display(train_plan)

## Save output results
manpower.write("manpower-planning-output.json")
manpower.write("manpower-planning-output.sol")


* Hiring plan's solution detail 
      s1     s2     s3
1  0.0    0.0   60.0
2  0.0  798.0  495.0
3  0.0  799.0  500.0
___________________________________
* Layoff plan's solution detail 
        s1   s2   s3
1  814.0  0.0  0.0
2  252.0  0.0  0.0
3  347.0  0.0  0.0
___________________________________
* Parttime plan's solution detail 
      s1   s2   s3
1  0.0  0.0  2.0
2  0.0  0.0  0.0
3  0.0  0.0  0.0
___________________________________
* Overmanning plan's solution detail 
      s1   s2   s3
1  0.0  0.0  0.0
2  0.0  0.0  0.0
3  0.0  0.0  0.0
___________________________________
* * Retraining detail plan: 
     s1 to s2  s1 to s3  s2 to s1  s2 to s3  s3 to s1  s3 to s2
1       0.0       0.0      25.0       0.0       3.0       0.0
2     148.0       0.0       0.0     109.0       0.0       0.0
3     104.0       0.0       2.0     140.0       0.0       8.0
* Retraining detail plan:


,s1 to s2,s1 to s3,s2 to s1,s2 to s3,s3 to s1,s3 to s2
1,0.0,0.0,25.0,0.0,3.0,0.0
2,148.0,0.0,0.0,109.0,0.0,0.0
3,104.0,0.0,2.0,140.0,0.0,8.0


In [48]:
789+495

1284

In [47]:
manpower.write("manpower-planning-output.json")
manpower.write("manpower-planning-output.sol")
